# Lesson 2: BigQuery Deep Dive - Practice Notebook

This notebook provides hands-on practice for BigQuery optimization techniques, window functions, and feature engineering for MLB data.

**Prerequisites:**
- Google Cloud project with BigQuery enabled
- MLB data in BigQuery dataset
- `google-cloud-bigquery` Python library installed

**What you'll learn:**
- Query optimization techniques
- Partitioning and clustering
- Window functions for rolling stats
- CTEs for complex queries
- Materialized views for features

## Setup and Configuration

First, let's install and import the necessary libraries.

In [1]:
# Install required packages (run once)
!pip install google-cloud-bigquery pandas db-dtypes pyarrow

In [2]:
# Import libraries
from google.cloud import bigquery
import pandas as pd
from datetime import datetime, timedelta
import os

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


### Configure BigQuery Connection

Update these settings to match your Google Cloud project:

In [3]:
# BigQuery Configuration
PROJECT_ID = "hankstank"
DATASET = "mlb_historical_data"

# Initialize BigQuery client
client = bigquery.Client(project=PROJECT_ID)

print(f"🔧 Connected to project: {PROJECT_ID}")
print(f"📊 Using dataset: {DATASET}")

🔧 Connected to project: hankstank
📊 Using dataset: mlb_historical_data


### Test Connection and Check Authentication

Let's verify you're authenticated and can access your dataset:

In [4]:
# Test connection and list tables in dataset
try:
    dataset_ref = client.dataset(DATASET)
    tables = list(client.list_tables(dataset_ref))
    
    print(f"✅ Successfully connected to {PROJECT_ID}.{DATASET}")
    print(f"\n📋 Available tables ({len(tables)}):")
    for table in tables:
        print(f"   - {table.table_id}")
except Exception as e:
    print(f"❌ Error connecting to BigQuery: {e}")
    print("\n💡 You may need to authenticate. Run this in terminal:")
    print("   gcloud auth application-default login")

✅ Successfully connected to hankstank.mlb_historical_data

📋 Available tables (9):
   - games_historical
   - games_historical_backup_20251224_112327
   - player_stats_historical
   - rosters_historical
   - standings_historical
   - statcast_pitches
   - team_stats_historical
   - teams_historical
   - transactions_historical


### Helper Functions

These functions make it easier to run queries and see results:

In [5]:
def run_query(query, show_cost=True, limit=10):
    """
    Execute a BigQuery query and return results as a DataFrame
    
    Args:
        query: SQL query string
        show_cost: Whether to show estimated query cost
        limit: Number of rows to display (None for all)
    
    Returns:
        pandas DataFrame with results
    """
    try:
        # Get query plan to estimate cost
        if show_cost:
            job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
            dry_run_job = client.query(query, job_config=job_config)
            bytes_processed = dry_run_job.total_bytes_processed
            gb_processed = bytes_processed / 1e9
            cost_estimate = (bytes_processed / 1e12) * 5  # $5 per TB
            
            print(f"📊 Query will process: {gb_processed:.3f} GB")
            print(f"💰 Estimated cost: ${cost_estimate:.6f}")
            print()
        
        # Run the actual query
        df = client.query(query).to_dataframe()
        
        print(f"✅ Query returned {len(df)} rows")
        
        # Display limited results
        if limit and len(df) > limit:
            print(f"   (showing first {limit} rows)\n")
            return df.head(limit)
        
        return df
        
    except Exception as e:
        print(f"❌ Query error: {e}")
        return None


def get_table_info(table_name):
    """Get information about a table's structure and size"""
    table_ref = client.dataset(DATASET).table(table_name)
    table = client.get_table(table_ref)
    
    print(f"📋 Table: {table_name}")
    print(f"   Rows: {table.num_rows:,}")
    print(f"   Size: {table.num_bytes / 1e9:.2f} GB")
    print(f"   Created: {table.created}")
    
    if table.time_partitioning:
        print(f"   Partitioned by: {table.time_partitioning.field}")
    else:
        print(f"   Partitioned: No")
    
    if table.clustering_fields:
        print(f"   Clustered by: {', '.join(table.clustering_fields)}")
    else:
        print(f"   Clustered: No")
    
    return table


print("✅ Helper functions loaded")

✅ Helper functions loaded


---

## Section 1: Query Optimization

Let's compare optimized vs unoptimized queries to see the difference in cost and performance.

### ❌ Bad Query: SELECT *

In [13]:
# Bad: Scans all columns (expensive!)
bad_query = f"""
SELECT * 
FROM `{PROJECT_ID}.{DATASET}.games_historical`
WHERE season = 2025
LIMIT 10
"""

result = run_query(bad_query)

📊 Query will process: 0.005 GB
💰 Estimated cost: $0.000023

✅ Query returned 10 rows


/Users/VTNX82W/Documents/personalDev/hanks_tank_ml/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


### ✅ Good Query: Select Only Needed Columns

In [14]:
# Good: Only scan needed columns (cheap!)
good_query = f"""
SELECT 
    game_pk,
    game_date,
    home_team_id,
    away_team_id,
    home_score,
    away_score,
    winning_team_id
FROM `{PROJECT_ID}.{DATASET}.games_historical`
WHERE season = 2025
ORDER BY game_date DESC
LIMIT 10
"""

result = run_query(good_query)
display(result)

📊 Query will process: 0.002 GB
💰 Estimated cost: $0.000009

✅ Query returned 10 rows


,game_pk,game_date,home_team_id,away_team_id,home_score,away_score,winning_team_id
0,813024,2025-11-01,141,119,4.0,5.0,119.0
1,813025,2025-10-31,141,119,1.0,3.0,119.0
2,813022,2025-10-29,119,141,1.0,6.0,141.0
3,813023,2025-10-28,119,141,2.0,6.0,141.0
4,813032,2025-10-27,119,141,6.0,5.0,119.0
5,813026,2025-10-25,141,119,1.0,5.0,119.0
6,813027,2025-10-24,141,119,11.0,4.0,141.0
7,813037,2025-10-20,141,136,4.0,3.0,141.0
8,813038,2025-10-19,141,136,6.0,2.0,141.0
9,813031,2025-10-17,119,158,5.0,1.0,119.0


### Check Table Structure

Let's see if your tables are partitioned and clustered:

In [15]:
# Check your main tables
tables_to_check = ['games_historical', 'team_stats_historical', 'statcast_pitches']

for table_name in tables_to_check:
    try:
        print("\n" + "="*60)
        get_table_info(table_name)
    except Exception as e:
        print(f"⚠️  Table {table_name} not found or inaccessible")


📋 Table: games_historical
   Rows: 27,172
   Size: 0.00 GB
   Created: 2025-12-24 16:23:31.853000+00:00
   Partitioned: No
   Clustered: No

📋 Table: team_stats_historical
   Rows: 700
   Size: 0.00 GB
   Created: 2025-08-19 19:35:56.286000+00:00
   Partitioned: No
   Clustered: No

📋 Table: statcast_pitches
   Rows: 7,813,531
   Size: 5.55 GB
   Created: 2025-12-19 16:59:49.336000+00:00
   Partitioned by: game_date
   Clustered by: year, game_pk, pitcher, batter


Analyze Current Query Performance
Run this query to see your most expensive tables:

In [26]:
query_expense_query = f"""
SELECT 
  ref_table.table_id,
  SUM(total_bytes_processed) / 1e12 AS total_tb_processed,
  COUNT(*) AS query_count,
  SUM(total_bytes_processed) / 1e12 * 5 AS estimated_cost_usd
FROM `region-us`.INFORMATION_SCHEMA.JOBS_BY_PROJECT,
  UNNEST(referenced_tables) AS ref_table
WHERE DATE(creation_time) >= DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)
  AND ref_table.table_id IS NOT NULL
GROUP BY ref_table.table_id
ORDER BY total_tb_processed DESC
LIMIT 20
"""

result = run_query(query_expense_query)
display(result)

📊 Query will process: 0.005 GB
💰 Estimated cost: $0.000025



/Users/VTNX82W/Documents/personalDev/hanks_tank_ml/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


✅ Query returned 10 rows


,table_id,total_tb_processed,query_count,estimated_cost_usd
0,statcast_pitches,0.009959,679,0.049793
1,games_historical,0.000167,125,0.000837
2,transactions_historical,0.000120,90,0.000602
3,rosters_historical,0.000093,57,0.000466
4,player_stats_historical,0.000073,38,0.000366
5,team_stats_historical,0.000072,54,0.000358
6,standings_historical,0.000071,50,0.000357
7,teams_historical,0.000071,24,0.000355
8,games_historical_deduped,0.000005,3,0.000025
9,__TABLES__,NaN,1,NaN


---

## Section 2: Window Functions

Window functions are essential for calculating rolling stats, rankings, and time-series features.

### Example 1: Rolling Win Percentage

Calculate each team's rolling 10-game win percentage:

In [27]:
rolling_wins_query = f"""
WITH team_games AS (
  SELECT 
    game_date,
    home_team_id AS team_id,
    CASE WHEN winning_team_id = home_team_id THEN 1 ELSE 0 END AS won
  FROM `{PROJECT_ID}.{DATASET}.games_historical`
  WHERE season = 2025
  
  UNION ALL
  
  SELECT 
    game_date,
    away_team_id AS team_id,
    CASE WHEN winning_team_id = away_team_id THEN 1 ELSE 0 END AS won
  FROM `{PROJECT_ID}.{DATASET}.games_historical`
  WHERE season = 2025
)
SELECT 
  team_id,
  game_date,
  won,
  AVG(won) OVER (
    PARTITION BY team_id
    ORDER BY game_date
    ROWS BETWEEN 9 PRECEDING AND CURRENT ROW
  ) AS rolling_10_win_pct,
  SUM(won) OVER (
    PARTITION BY team_id
    ORDER BY game_date
  ) AS season_wins
FROM team_games
ORDER BY team_id, game_date DESC
LIMIT 50
"""

result = run_query(rolling_wins_query)
display(result)

📊 Query will process: 0.001 GB
💰 Estimated cost: $0.000005



/Users/VTNX82W/Documents/personalDev/hanks_tank_ml/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


✅ Query returned 50 rows
   (showing first 10 rows)



,team_id,game_date,won,rolling_10_win_pct,season_wins
0,108,2025-09-28,0,0.3,72
1,108,2025-09-27,0,0.3,72
2,108,2025-09-26,1,0.3,72
3,108,2025-09-25,0,0.2,71
4,108,2025-09-24,1,0.2,71
5,108,2025-09-23,0,0.1,70
6,108,2025-09-21,0,0.1,70
7,108,2025-09-20,1,0.2,70
8,108,2025-09-19,0,0.2,69
9,108,2025-09-18,0,0.2,69


### Example 2: LAG Function - Compare to Previous Game

In [21]:
lag_query = f"""
SELECT 
  game_date,
  home_team_id,
  home_score,
  away_score,
  LAG(home_score, 1) OVER (PARTITION BY home_team_id ORDER BY game_date) AS prev_home_score,
  LAG(game_date, 1) OVER (PARTITION BY home_team_id ORDER BY game_date) AS prev_game_date,
  DATE_DIFF(game_date, LAG(game_date, 1) OVER (PARTITION BY home_team_id ORDER BY game_date), DAY) AS days_rest
FROM `{PROJECT_ID}.{DATASET}.games_historical`
WHERE season = 2025 AND home_team_id = 144
ORDER BY game_date DESC
LIMIT 20
"""

result = run_query(lag_query)
display(result)

📊 Query will process: 0.001 GB
💰 Estimated cost: $0.000005



/Users/VTNX82W/Documents/personalDev/hanks_tank_ml/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


✅ Query returned 20 rows
   (showing first 10 rows)



,game_date,home_team_id,home_score,away_score,prev_home_score,prev_game_date,days_rest
0,2025-09-28,144,4.0,1.0,1.0,2025-09-27,1
1,2025-09-27,144,1.0,3.0,3.0,2025-09-26,1
2,2025-09-26,144,3.0,9.0,3.0,2025-09-24,2
3,2025-09-24,144,3.0,4.0,3.0,2025-09-23,1
4,2025-09-23,144,3.0,2.0,11.0,2025-09-22,1
5,2025-09-22,144,11.0,5.0,8.0,2025-09-14,8
6,2025-09-14,144,8.0,3.0,2.0,2025-09-13,1
7,2025-09-13,144,2.0,6.0,3.0,2025-09-12,1
8,2025-09-12,144,3.0,11.0,2.0,2025-09-10,2
9,2025-09-10,144,2.0,3.0,1.0,2025-09-09,1


---

## Section 3: Practice Queries

Now it's your turn! Try these exercises on your own.

### Exercise 1: Team Momentum Score

Calculate a momentum score combining win rate and run differential over last 10 games:

In [29]:
# Your code here - calculate momentum score
# Hint: Use window functions with rolling averages
# Formula: 0.7 * win_rate_l10 + 0.3 * (run_diff_l10 / 40)

momentum_query = f"""
WITH team_games AS (
  SELECT 
    game_date,
    home_team_id AS team_id,
    home_score AS runs_scored,
    away_score AS runs_allowed,
    CASE WHEN winning_team_id = home_team_id THEN 1 ELSE 0 END AS won
  FROM `{PROJECT_ID}.{DATASET}.games_historical`
  WHERE season = 2025
  
  UNION ALL
  
  SELECT 
    game_date,
    away_team_id AS team_id,
    away_score AS runs_scored,
    home_score AS runs_allowed,
    CASE WHEN winning_team_id = away_team_id THEN 1 ELSE 0 END AS won
  FROM `{PROJECT_ID}.{DATASET}.games_historical`
  WHERE season = 2025
)
SELECT 
  team_id,
  game_date,
  -- Win rate in last 10 games
  AVG(CAST(won AS FLOAT64)) OVER w10 AS win_rate_l10,
  -- Run differential in last 10 games
  SUM(runs_scored - runs_allowed) OVER w10 AS run_diff_l10,
  -- Momentum score (weighted average)
  0.7 * AVG(CAST(won AS FLOAT64)) OVER w10 + 
  0.3 * (SUM(runs_scored - runs_allowed) OVER w10 / 40.0) AS momentum_score
FROM team_games
WINDOW w10 AS (
  PARTITION BY team_id 
  ORDER BY game_date 
  ROWS BETWEEN 9 PRECEDING AND CURRENT ROW
)
ORDER BY game_date DESC, momentum_score DESC
LIMIT 50
"""

# Uncomment when ready:
result = run_query(momentum_query)
display(result)

📊 Query will process: 0.002 GB
💰 Estimated cost: $0.000008



/Users/VTNX82W/Documents/personalDev/hanks_tank_ml/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


✅ Query returned 50 rows
   (showing first 10 rows)



,team_id,game_date,win_rate_l10,run_diff_l10,momentum_score
0,119,2025-11-01,0.7,2.0,0.5050
1,141,2025-11-01,0.5,9.0,0.4175
2,141,2025-10-31,0.6,16.0,0.5400
3,119,2025-10-31,0.7,2.0,0.5050
4,141,2025-10-29,0.7,27.0,0.6925
5,119,2025-10-29,0.7,1.0,0.4975
6,141,2025-10-28,0.6,15.0,0.5325
7,119,2025-10-28,0.7,0.0,0.4900
8,119,2025-10-27,0.8,5.0,0.5975
9,141,2025-10-27,0.5,9.0,0.4175


### Exercise 2: Home vs Away Performance

Compare each team's home win percentage to their away win percentage:

In [31]:
# Your code here - calculate home vs away splits
# Hint: Use CASE statements and GROUP BY

home_away_query = f"""
WITH team_games AS (
    SELECT 
        game_date,
        home_team_id AS team_id,
        CASE WHEN winning_team_id = home_team_id THEN 1 ELSE 0 END AS won,
        TRUE AS is_home
    FROM `{PROJECT_ID}.{DATASET}.games_historical`
    WHERE season = 2025
    
    UNION ALL
    
    SELECT 
        game_date,
        away_team_id AS team_id,
        CASE WHEN winning_team_id = away_team_id THEN 1 ELSE 0 END AS won,
        FALSE AS is_home
    FROM `{PROJECT_ID}.{DATASET}.games_historical`
    WHERE season = 2025
),
team_splits AS (
    SELECT 
        team_id,
        SUM(CASE WHEN is_home THEN won ELSE 0 END) AS home_wins,
        COUNT(CASE WHEN is_home THEN 1 END) AS home_games,
        SUM(CASE WHEN NOT is_home THEN won ELSE 0 END) AS away_wins,
        COUNT(CASE WHEN NOT is_home THEN 1 END) AS away_games
    FROM team_games
    GROUP BY team_id
)
SELECT 
    team_id,
    home_wins,
    home_games,
    away_wins,
    away_games,
    ROUND(home_wins * 1.0 / NULLIF(home_games, 0), 3) AS home_win_pct,
    ROUND(away_wins * 1.0 / NULLIF(away_games, 0), 3) AS away_win_pct,
    ROUND((home_wins * 1.0 / NULLIF(home_games, 0)) - (away_wins * 1.0 / NULLIF(away_games, 0)), 3) AS home_advantage
FROM team_splits
WHERE home_games > 0 AND away_games > 0
ORDER BY home_advantage DESC
"""

# Uncomment when ready:
result = run_query(home_away_query)
display(result)

📊 Query will process: 0.001 GB
💰 Estimated cost: $0.000004



/Users/VTNX82W/Documents/personalDev/hanks_tank_ml/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


✅ Query returned 30 rows
   (showing first 10 rows)



,team_id,home_wins,home_games,away_wins,away_games,home_win_pct,away_win_pct,home_advantage
0,134,44,81,27,81,0.543,0.333,0.210
1,140,48,81,33,81,0.593,0.407,0.185
2,121,49,81,34,81,0.605,0.420,0.185
3,135,52,81,39,84,0.642,0.464,0.178
4,143,55,83,42,83,0.663,0.506,0.157
5,141,59,91,45,89,0.648,0.506,0.143
6,136,54,87,42,87,0.621,0.483,0.138
7,112,54,86,42,84,0.628,0.500,0.128
8,138,44,81,34,81,0.543,0.420,0.123
9,158,55,86,45,85,0.640,0.529,0.110


### Exercise 3: Your Own Query!

Write a query to answer a question you're curious about in the data:

In [ ]:
# Your custom query here
# Ideas:
# - Which teams score the most runs in close games?
# - What's the average score differential by month?
# - Which teams have the longest win streaks?

custom_query = f"""
-- Write your query here

"""

# Uncomment when ready:
# result = run_query(custom_query)
# display(result)

---

## Section 4: Advanced - CTEs and Feature Engineering

Use CTEs to build complex feature sets for machine learning:

In [38]:
feature_engineering_query = f"""
-- Step 1: Get team stats per game
WITH team_games AS (
  SELECT 
    game_pk AS game_id,
    game_date,
    home_team_id AS team_id,
    home_score AS runs_scored,
    away_score AS runs_allowed,
    CASE WHEN winning_team_id = home_team_id THEN 1 ELSE 0 END AS won
  FROM `{PROJECT_ID}.{DATASET}.games_historical`
  WHERE season = 2025
  
  UNION ALL
  
  SELECT 
    game_pk AS game_id,
    game_date,
    away_team_id AS team_id,
    away_score AS runs_scored,
    home_score AS runs_allowed,
    CASE WHEN winning_team_id = away_team_id THEN 1 ELSE 0 END AS won
  FROM `{PROJECT_ID}.{DATASET}.games_historical`
  WHERE season = 2025
),

-- Step 2: Calculate rolling statistics
team_rolling AS (
  SELECT 
    team_id,
    game_date,
    game_id,
    runs_scored,
    runs_allowed,
    won,
    AVG(won) OVER w10 AS l10_win_pct,
    AVG(runs_scored) OVER w10 AS l10_runs_scored,
    AVG(runs_allowed) OVER w10 AS l10_runs_allowed,
    SUM(won) OVER season AS season_wins,
    COUNT(*) OVER season AS games_played
  FROM team_games
  WINDOW 
    w10 AS (PARTITION BY team_id ORDER BY game_date ROWS BETWEEN 9 PRECEDING AND CURRENT ROW),
    season AS (PARTITION BY team_id ORDER BY game_date ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
)

-- Step 3: Get current features for each team
SELECT 
  team_id,
  game_date,
  l10_win_pct,
  l10_runs_scored,
  l10_runs_allowed,
  l10_runs_scored - l10_runs_allowed AS l10_run_diff,
  season_wins,
  games_played,
  season_wins * 1.0 / games_played AS season_win_pct
FROM team_rolling
ORDER BY game_date DESC, l10_win_pct DESC
LIMIT 30
"""

result = run_query(feature_engineering_query)
display(result)

📊 Query will process: 0.002 GB
💰 Estimated cost: $0.000008

✅ Query returned 30 rows
   (showing first 10 rows)



/Users/VTNX82W/Documents/personalDev/hanks_tank_ml/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,team_id,game_date,l10_win_pct,l10_runs_scored,l10_runs_allowed,l10_run_diff,season_wins,games_played,season_win_pct
0,119,2025-11-01,0.7,3.9,3.7,0.2,106,179,0.592179
1,141,2025-11-01,0.5,4.6,3.7,0.9,104,180,0.577778
2,119,2025-10-31,0.7,3.6,3.4,0.2,105,178,0.589888
3,141,2025-10-31,0.6,5.0,3.4,1.6,104,179,0.581006
4,119,2025-10-29,0.7,3.5,3.4,0.1,104,177,0.587571
5,141,2025-10-29,0.7,6.2,3.5,2.7,104,178,0.584270
6,119,2025-10-28,0.7,3.6,3.6,0.0,104,176,0.590909
7,141,2025-10-28,0.6,5.9,4.4,1.5,103,177,0.581921
8,119,2025-10-27,0.8,3.8,3.3,0.5,104,175,0.594286
9,141,2025-10-27,0.5,5.4,4.5,0.9,102,176,0.579545


In [39]:
# Check what columns exist in games_historical table
schema_query = f"""
SELECT column_name, data_type
FROM `{PROJECT_ID}.{DATASET}.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'games_historical'
ORDER BY ordinal_position
"""

print("Checking table schema...")
schema_result = run_query(schema_query, show_cost=False, limit=None)
if schema_result is not None:
    display(schema_result)

Checking table schema...


/Users/VTNX82W/Documents/personalDev/hanks_tank_ml/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


✅ Query returned 19 rows


,column_name,data_type
0,year,INT64
1,game_pk,INT64
2,game_date,DATE
3,game_datetime,TIMESTAMP
4,status_code,STRING
5,status_description,STRING
6,home_team_id,INT64
7,home_team_name,STRING
8,away_team_id,INT64
9,away_team_name,STRING


In [40]:
# Let's see what data actually exists - peek at the table
peek_query = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET}.games_historical`
LIMIT 5
"""

print("\nPeeking at first 5 rows of games_historical:")
peek_result = run_query(peek_query, show_cost=False, limit=None)
if peek_result is not None:
    display(peek_result)


Peeking at first 5 rows of games_historical:


/Users/VTNX82W/Documents/personalDev/hanks_tank_ml/.venv/lib/python3.13/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


✅ Query returned 5 rows


,year,game_pk,game_date,game_datetime,status_code,status_description,home_team_id,home_team_name,away_team_id,away_team_name,venue_id,venue_name,game_type,season,season_display,home_score,away_score,winning_team_id,losing_team_id
0,2015,414418,2015-06-01,2015-06-02 02:15:00+00:00,F,Final,137,San Francisco Giants,134,Pittsburgh Pirates,2395,AT&T Park,R,2015,2015,3.0,4.0,134.0,137.0
1,2015,414610,2015-06-15,2015-06-16 02:15:00+00:00,F,Final,137,San Francisco Giants,136,Seattle Mariners,2395,AT&T Park,R,2015,2015,1.0,5.0,136.0,137.0
2,2015,416054,2015-10-03,2015-10-03 20:05:00+00:00,F,Final,137,San Francisco Giants,115,Colorado Rockies,2395,AT&T Park,R,2015,2015,3.0,2.0,137.0,115.0
3,2015,414381,2015-05-30,2015-05-31 02:05:00+00:00,F,Final,137,San Francisco Giants,144,Atlanta Braves,2395,AT&T Park,R,2015,2015,0.0,8.0,144.0,137.0
4,2015,415759,2015-09-11,2015-09-12 02:15:00+00:00,F,Final,137,San Francisco Giants,135,San Diego Padres,2395,AT&T Park,R,2015,2015,9.0,1.0,137.0,135.0
